In [11]:
import csv
import math
import pandas as pd
import numpy as np

In [12]:
def std_dev(distr, avg, num_samples):
    sum_of_squares = 0
    for k,v in distr.items():
        sum_of_squares += v*((k-avg)**2)
    variance = sum_of_squares/num_samples
    return math.sqrt(variance)

In [13]:
training_df = pd.read_csv('training.csv')
training_df.drop('article_number', axis=1, inplace=True)
print(training_df)

                                          article_words  \
0     open,absent,cent,cent,cent,stock,inflow,rate,k...   
1     morn,stead,end,end,day,day,day,patch,patch,pat...   
2     socc,socc,world,world,recent,law,fifa,fifa,fif...   
3     open,forint,forint,forint,forint,cent,cent,ste...   
4     morn,complet,weekend,minut,minut,minut,arrow,d...   
5     regist,equal,stock,stock,city,city,period,issu...   
6     cumulat,record,month,bank,balanc,cash,jan,jan,...   
7     divid,ten,rule,weekend,social,sweep,destruc,su...   
8     hit,regist,publ,gmt,count,week,fall,articl,art...   
9     queen,queen,park,hit,minist,minist,minist,mini...   
10    contrast,research,heavy,heavy,weather,sud,mirr...   
11    weather,underl,half,half,half,half,half,half,h...   
12    portug,portug,rate,fix,money,escudo,drain,drai...   
13    open,shekel,shekel,shekel,shekel,shekel,shekel...   
14    mgt,conc,praecip,limit,emtn,bp,year,full,und,m...   
15    stabl,stabl,strength,cent,cent,cent,hagstrom,q... 

In [14]:
labels = training_df['topic']
label_distr = labels.value_counts()
print(label_distr)
print(label_distr.shape)
print(label_distr['IRRELEVANT'])
label_sum = label_distr.sum()
print(label_sum)
label_distr = pd.concat([pd.Series(data=[label_sum], index=['global']), label_distr])
print(label_distr)
print(label_distr['DEFENCE'])

IRRELEVANT                          4734
MONEY MARKETS                       1673
SPORTS                              1102
FOREX MARKETS                        845
DEFENCE                              258
SHARE LISTINGS                       218
HEALTH                               183
BIOGRAPHIES PERSONALITIES PEOPLE     167
DOMESTIC MARKETS                     133
ARTS CULTURE ENTERTAINMENT           117
SCIENCE AND TECHNOLOGY                70
Name: topic, dtype: int64
(11,)
4734
9500
global                              9500
IRRELEVANT                          4734
MONEY MARKETS                       1673
SPORTS                              1102
FOREX MARKETS                        845
DEFENCE                              258
SHARE LISTINGS                       218
HEALTH                               183
BIOGRAPHIES PERSONALITIES PEOPLE     167
DOMESTIC MARKETS                     133
ARTS CULTURE ENTERTAINMENT           117
SCIENCE AND TECHNOLOGY                70
dtype: int64
25

In [15]:
distr = {'global':{}}

for row in training_df.itertuples():
    sentence = row.article_words
    sentence = sentence.split(sep=',')
    words = len(sentence)
    if words not in distr['global']:
        distr['global'][words] = 0
    distr['global'][words] += 1
    #get label counts from dataframe
#     if 'count' not in distr['global']:
#         distr['global']['count'] = 0
#     distr['global']['count'] += 1
    if 'total' not in distr['global']:
        distr['global']['total'] = 0
    distr['global']['total'] += words
    #topic = row.topic.lower()
    topic = row.topic
    if topic not in distr:
        distr[topic] = {}
    if words not in distr[topic]:
        distr[topic][words] = 0
    distr[topic][words] += 1
    if 'total' not in distr[topic]:
        distr[topic]['total'] = 0
    distr[topic]['total'] += words

print(distr)

{'global': {84: 63, 'total': 1213216, 61: 62, 59: 66, 67: 59, 76: 51, 48: 58, 119: 33, 149: 26, 306: 10, 158: 23, 17: 18, 131: 33, 129: 30, 71: 75, 285: 10, 27: 43, 233: 21, 300: 15, 46: 85, 265: 15, 68: 51, 120: 36, 49: 69, 85: 51, 51: 68, 399: 8, 124: 35, 26: 43, 126: 38, 198: 16, 93: 55, 43: 59, 159: 22, 66: 40, 168: 25, 91: 63, 257: 14, 181: 20, 199: 18, 13: 23, 72: 48, 204: 13, 74: 58, 298: 8, 79: 50, 275: 13, 175: 16, 346: 5, 78: 64, 296: 7, 282: 15, 24: 54, 111: 52, 34: 58, 81: 51, 294: 13, 92: 54, 109: 39, 63: 48, 150: 27, 54: 62, 19: 40, 140: 29, 53: 77, 73: 70, 30: 49, 317: 6, 155: 30, 98: 50, 246: 20, 88: 49, 122: 49, 55: 67, 47: 80, 23: 40, 137: 30, 171: 16, 202: 15, 139: 25, 94: 54, 28: 50, 69: 53, 56: 80, 104: 48, 35: 78, 196: 20, 266: 13, 96: 32, 22: 67, 405: 3, 107: 43, 272: 11, 162: 23, 132: 26, 58: 75, 160: 24, 57: 58, 241: 17, 123: 30, 33: 61, 99: 40, 151: 31, 141: 21, 172: 31, 232: 17, 38: 71, 211: 19, 95: 41, 113: 37, 197: 11, 127: 26, 230: 15, 118: 30, 273: 17, 52

In [16]:
##messy as hell
print('text length statistics')
for label, lengths in distr.items():
    num_samples = label_distr[label]
    avg = lengths['total']/num_samples
    del lengths['total'] #total should be separate...not in the distr data...classes might be easier
    filtered_lengths = [x for x in lengths.keys() if isinstance(x, int)]
    #print(filtered_lengths)
    deviation = std_dev(lengths, avg, num_samples)
    min_len = min(filtered_lengths)
    max_len = max(filtered_lengths)
    freqs = [v for k,v in lengths.items() if k != 'total']
    mode = max(freqs)
    modes = [k for k,v in lengths.items() if v == mode]
    coeff_variation = deviation/avg
    print(label, ":")
    print('\tavg: ', avg)
    print('\tstd_dev:', deviation)
    print('\tcoef_var:', coeff_variation)
    print('\tmin: ', min_len) 
    print('\tmax: ', max_len)
    print('\tmode: ', mode, modes)

text length statistics
global :
	avg:  127.70694736842105
	std_dev: 98.74762096613219
	coef_var: 0.77323609248333
	min:  8
	max:  1935
	mode:  101 [32]
FOREX MARKETS :
	avg:  136.02958579881656
	std_dev: 104.12665399131035
	coef_var: 0.7654706391983753
	min:  13
	max:  532
	mode:  15 [22]
MONEY MARKETS :
	avg:  110.89958158995816
	std_dev: 92.81054599302647
	coef_var: 0.8368881528893917
	min:  13
	max:  546
	mode:  28 [45]
SPORTS :
	avg:  127.38929219600726
	std_dev: 99.84094425793315
	coef_var: 0.7837467540388959
	min:  12
	max:  769
	mode:  15 [52]
IRRELEVANT :
	avg:  125.29404309252217
	std_dev: 98.80569313658468
	coef_var: 0.7885905083581872
	min:  8
	max:  1935
	mode:  50 [36]
SHARE LISTINGS :
	avg:  95.38990825688073
	std_dev: 79.06581446758628
	coef_var: 0.8288698030263915
	min:  14
	max:  616
	mode:  7 [67]
BIOGRAPHIES PERSONALITIES PEOPLE :
	avg:  185.311377245509
	std_dev: 101.08180739740433
	coef_var: 0.5454700563985693
	min:  34
	max:  626
	mode:  5 [85]
DOMESTIC MARKETS :


In [17]:
#corpus building
corpus = {}
for row in training_df.itertuples():
    words = row.article_words.split(',')
    for word in words:
        if word not in corpus:
            corpus[word] = {}
        if 'global' not in corpus[word]:
            corpus[word]['global'] = 0
        corpus[word]['global'] += 1
        if row.topic not in corpus[word]:
            corpus[word][row.topic] = 0
        corpus[word][row.topic] += 1
#print(corpus)

for word, word_dist in corpus.items():
    print(word)
    for topic, freq in word_dist.items():
        print('\t', topic, freq)

open
	 global 2267
	 FOREX MARKETS 361
	 SPORTS 502
	 BIOGRAPHIES PERSONALITIES PEOPLE 37
	 MONEY MARKETS 528
	 IRRELEVANT 673
	 SCIENCE AND TECHNOLOGY 19
	 HEALTH 18
	 DEFENCE 65
	 SHARE LISTINGS 28
	 DOMESTIC MARKETS 5
	 ARTS CULTURE ENTERTAINMENT 31
absent
	 global 138
	 FOREX MARKETS 20
	 IRRELEVANT 54
	 MONEY MARKETS 25
	 SPORTS 30
	 DEFENCE 3
	 DOMESTIC MARKETS 1
	 HEALTH 2
	 BIOGRAPHIES PERSONALITIES PEOPLE 2
	 ARTS CULTURE ENTERTAINMENT 1
cent
	 global 4987
	 FOREX MARKETS 953
	 MONEY MARKETS 1533
	 SPORTS 158
	 IRRELEVANT 2049
	 SCIENCE AND TECHNOLOGY 25
	 HEALTH 45
	 DEFENCE 67
	 BIOGRAPHIES PERSONALITIES PEOPLE 70
	 DOMESTIC MARKETS 22
	 SHARE LISTINGS 41
	 ARTS CULTURE ENTERTAINMENT 24
stock
	 global 2637
	 FOREX MARKETS 342
	 SHARE LISTINGS 282
	 IRRELEVANT 1539
	 MONEY MARKETS 416
	 DEFENCE 7
	 DOMESTIC MARKETS 19
	 ARTS CULTURE ENTERTAINMENT 3
	 HEALTH 14
	 BIOGRAPHIES PERSONALITIES PEOPLE 2
	 SPORTS 13
inflow
	 global 194
	 FOREX MARKETS 65
	 IRRELEVANT 42
	 MONEY MARKE

	 DEFENCE 49
	 SHARE LISTINGS 40
	 HEALTH 18
	 SCIENCE AND TECHNOLOGY 10
	 DOMESTIC MARKETS 10
	 ARTS CULTURE ENTERTAINMENT 22
test
	 global 741
	 FOREX MARKETS 45
	 IRRELEVANT 188
	 SPORTS 308
	 SCIENCE AND TECHNOLOGY 19
	 MONEY MARKETS 54
	 HEALTH 54
	 BIOGRAPHIES PERSONALITIES PEOPLE 26
	 DEFENCE 44
	 ARTS CULTURE ENTERTAINMENT 1
	 DOMESTIC MARKETS 1
	 SHARE LISTINGS 1
mid
	 global 421
	 FOREX MARKETS 81
	 MONEY MARKETS 90
	 SPORTS 22
	 IRRELEVANT 187
	 SHARE LISTINGS 8
	 BIOGRAPHIES PERSONALITIES PEOPLE 8
	 HEALTH 2
	 SCIENCE AND TECHNOLOGY 4
	 ARTS CULTURE ENTERTAINMENT 6
	 DOMESTIC MARKETS 4
	 DEFENCE 9
sale
	 global 2378
	 FOREX MARKETS 149
	 IRRELEVANT 1758
	 MONEY MARKETS 228
	 HEALTH 33
	 DEFENCE 50
	 DOMESTIC MARKETS 73
	 SPORTS 21
	 ARTS CULTURE ENTERTAINMENT 18
	 SHARE LISTINGS 38
	 SCIENCE AND TECHNOLOGY 3
	 BIOGRAPHIES PERSONALITIES PEOPLE 7
compar
	 global 1206
	 FOREX MARKETS 195
	 IRRELEVANT 588
	 MONEY MARKETS 320
	 BIOGRAPHIES PERSONALITIES PEOPLE 6
	 SHARE LISTINGS

	 global 710
	 SPORTS 52
	 IRRELEVANT 528
	 SHARE LISTINGS 48
	 DOMESTIC MARKETS 10
	 FOREX MARKETS 12
	 MONEY MARKETS 13
	 BIOGRAPHIES PERSONALITIES PEOPLE 10
	 ARTS CULTURE ENTERTAINMENT 23
	 DEFENCE 7
	 HEALTH 5
	 SCIENCE AND TECHNOLOGY 2
brazil
	 global 642
	 SPORTS 127
	 IRRELEVANT 339
	 DOMESTIC MARKETS 44
	 FOREX MARKETS 38
	 DEFENCE 11
	 BIOGRAPHIES PERSONALITIES PEOPLE 9
	 MONEY MARKETS 63
	 HEALTH 5
	 ARTS CULTURE ENTERTAINMENT 5
	 SCIENCE AND TECHNOLOGY 1
footbal
	 global 172
	 SPORTS 139
	 IRRELEVANT 22
	 BIOGRAPHIES PERSONALITIES PEOPLE 9
	 SCIENCE AND TECHNOLOGY 1
	 ARTS CULTURE ENTERTAINMENT 1
body
	 global 275
	 SPORTS 27
	 IRRELEVANT 149
	 MONEY MARKETS 7
	 HEALTH 26
	 DOMESTIC MARKETS 5
	 DEFENCE 18
	 SHARE LISTINGS 2
	 BIOGRAPHIES PERSONALITIES PEOPLE 27
	 FOREX MARKETS 4
	 SCIENCE AND TECHNOLOGY 4
	 ARTS CULTURE ENTERTAINMENT 6
confus
	 global 62
	 SPORTS 6
	 IRRELEVANT 25
	 SCIENCE AND TECHNOLOGY 3
	 FOREX MARKETS 9
	 SHARE LISTINGS 2
	 MONEY MARKETS 13
	 HEALTH 1


	 DEFENCE 2
	 BIOGRAPHIES PERSONALITIES PEOPLE 1
clear
	 global 637
	 FOREX MARKETS 72
	 BIOGRAPHIES PERSONALITIES PEOPLE 15
	 MONEY MARKETS 72
	 DEFENCE 50
	 IRRELEVANT 321
	 SPORTS 60
	 HEALTH 24
	 DOMESTIC MARKETS 8
	 SCIENCE AND TECHNOLOGY 8
	 ARTS CULTURE ENTERTAINMENT 3
	 SHARE LISTINGS 4
dlr
	 global 330
	 FOREX MARKETS 146
	 MONEY MARKETS 167
	 IRRELEVANT 17
futur
	 global 1454
	 FOREX MARKETS 185
	 IRRELEVANT 771
	 SCIENCE AND TECHNOLOGY 8
	 BIOGRAPHIES PERSONALITIES PEOPLE 20
	 SPORTS 51
	 MONEY MARKETS 322
	 DEFENCE 51
	 SHARE LISTINGS 22
	 DOMESTIC MARKETS 5
	 ARTS CULTURE ENTERTAINMENT 14
	 HEALTH 5
hung
	 global 501
	 FOREX MARKETS 56
	 IRRELEVANT 215
	 MONEY MARKETS 132
	 SPORTS 30
	 DEFENCE 43
	 BIOGRAPHIES PERSONALITIES PEOPLE 12
	 HEALTH 7
	 SHARE LISTINGS 3
	 ARTS CULTURE ENTERTAINMENT 3
complet
	 global 523
	 IRRELEVANT 304
	 BIOGRAPHIES PERSONALITIES PEOPLE 5
	 SPORTS 83
	 HEALTH 15
	 MONEY MARKETS 37
	 FOREX MARKETS 21
	 ARTS CULTURE ENTERTAINMENT 10
	 DEFENCE 23


	 global 848
	 SHARE LISTINGS 14
	 FOREX MARKETS 61
	 IRRELEVANT 496
	 MONEY MARKETS 120
	 DOMESTIC MARKETS 40
	 DEFENCE 12
	 BIOGRAPHIES PERSONALITIES PEOPLE 13
	 SPORTS 70
	 SCIENCE AND TECHNOLOGY 3
	 ARTS CULTURE ENTERTAINMENT 7
	 HEALTH 12
issu
	 global 2417
	 SHARE LISTINGS 133
	 FOREX MARKETS 107
	 IRRELEVANT 1326
	 MONEY MARKETS 541
	 BIOGRAPHIES PERSONALITIES PEOPLE 45
	 DEFENCE 108
	 HEALTH 65
	 SPORTS 30
	 DOMESTIC MARKETS 29
	 ARTS CULTURE ENTERTAINMENT 24
	 SCIENCE AND TECHNOLOGY 9
bursatil
	 global 1
	 SHARE LISTINGS 1
dan
	 global 59
	 SHARE LISTINGS 1
	 FOREX MARKETS 13
	 HEALTH 2
	 MONEY MARKETS 6
	 IRRELEVANT 28
	 SPORTS 8
	 BIOGRAPHIES PERSONALITIES PEOPLE 1
million
	 global 7034
	 SHARE LISTINGS 296
	 IRRELEVANT 4740
	 FOREX MARKETS 304
	 BIOGRAPHIES PERSONALITIES PEOPLE 60
	 MONEY MARKETS 920
	 DOMESTIC MARKETS 152
	 SPORTS 216
	 DEFENCE 94
	 HEALTH 126
	 SCIENCE AND TECHNOLOGY 24
	 ARTS CULTURE ENTERTAINMENT 102
gbm
	 global 2
	 SHARE LISTINGS 2
capit
	 global 1190

...

	 SPORTS 1
tinbergen
	 global 1
	 MONEY MARKETS 1
menan
	 global 1
	 IRRELEVANT 1
blackfoot
	 global 1
	 IRRELEVANT 1
bingham
	 global 2
	 IRRELEVANT 2
snowpack
	 global 1
	 IRRELEVANT 1
chaiyawat
	 global 4
	 IRRELEVANT 4
wibulswasd
	 global 1
	 IRRELEVANT 1
ganjarernd
	 global 1
	 IRRELEVANT 1
cory
	 global 2
	 IRRELEVANT 1
	 SPORTS 1
stumpel
	 global 2
	 SPORTS 2
laperrier
	 global 2
	 SPORTS 2
kolzig
	 global 2
	 SPORTS 2
giguer
	 global 1
	 SPORTS 1
larionov
	 global 1
	 SPORTS 1
alow
	 global 1
	 SPORTS 1
tsyplakov
	 global 1
	 SPORTS 1
bowm
	 global 1
	 SPORTS 1
vopat
	 global 1
	 SPORTS 1
devorsk
	 global 1
	 SPORTS 1
poulin
	 global 1
	 SPORTS 1
primeau
	 global 1
	 SPORTS 1
jarom
	 global 1
	 SPORTS 1
iginl
	 global 1
	 SPORTS 1
quennevill
	 global 1
	 SPORTS 1
mctav
	 global 1
	 SPORTS 1
khristich
	 global 1
	 SPORTS 1
bodg
	 global 1
	 SPORTS 1
cottey
	 global 2
	 SPORTS 2
cosk
	 global 1
	 SPORTS 1
metson
	 global 1
	 SPORTS 1
dahab
	 global 1
	 IRRELEVANT 1
dakhl
	 global

	 global 2
	 IRRELEVANT 2
lohan
	 global 4
	 IRRELEVANT 4
comstock
	 global 3
	 IRRELEVANT 3
roquet
	 global 1
	 SHARE LISTINGS 1
desportiv
	 global 2
	 SHARE LISTINGS 2
poe
	 global 1
	 IRRELEVANT 1
recign
	 global 1
	 IRRELEVANT 1
unprecend
	 global 1
	 IRRELEVANT 1
buc
	 global 1
	 IRRELEVANT 1
saavedr
	 global 1
	 IRRELEVANT 1
flap
	 global 2
	 IRRELEVANT 2
lamontagn
	 global 1
	 IRRELEVANT 1
rindskopf
	 global 1
	 IRRELEVANT 1
corydon
	 global 1
	 IRRELEVANT 1
keren
	 global 1
	 IRRELEVANT 1
calhoun
	 global 1
	 IRRELEVANT 1
patton
	 global 1
	 IRRELEVANT 1
ripley
	 global 2
	 IRRELEVANT 2
bentonvill
	 global 1
	 IRRELEVANT 1
schaf
	 global 2
	 IRRELEVANT 2
anuradhapur
	 global 1
	 HEALTH 1
degroot
	 global 1
	 SPORTS 1
anguill
	 global 1
	 SPORTS 1
tobag
	 global 1
	 SPORTS 1
mcgarrel
	 global 1
	 SPORTS 1
whet
	 global 1
	 IRRELEVANT 1
primofin
	 global 1
	 IRRELEVANT 1
gpg
	 global 1
	 IRRELEVANT 1
rthk
	 global 3
	 IRRELEVANT 3
voyl
	 global 1
	 SPORTS 1
bg3017496006
	 global 

	 IRRELEVANT 1
leskin
	 global 4
	 IRRELEVANT 4
elak
	 global 2
	 IRRELEVANT 2
jouk
	 global 1
	 IRRELEVANT 1
shakt
	 global 1
	 IRRELEVANT 1
nang
	 global 1
	 IRRELEVANT 1
pathein
	 global 1
	 IRRELEVANT 1
parbat
	 global 1
	 IRRELEVANT 1
muriat
	 global 1
	 IRRELEVANT 1
primark
	 global 1
	 IRRELEVANT 1
leibfritz
	 global 1
	 IRRELEVANT 1
hulus
	 global 1
	 IRRELEVANT 1
tanm
	 global 1
	 IRRELEVANT 1
repsect
	 global 1
	 FOREX MARKETS 1
repteat
	 global 1
	 FOREX MARKETS 1
bg3011197004
	 global 1
	 MONEY MARKETS 1
banky
	 global 1
	 IRRELEVANT 1
bavel
	 global 1
	 IRRELEVANT 1
laboucher
	 global 1
	 IRRELEVANT 1
daan
	 global 1
	 IRRELEVANT 1
bossch
	 global 1
	 IRRELEVANT 1
lenght
	 global 1
	 SPORTS 1
moliv
	 global 1
	 SPORTS 1
strato
	 global 1
	 SPORTS 1
bradber
	 global 1
	 IRRELEVANT 1
blakesl
	 global 1
	 IRRELEVANT 1
transmontaign
	 global 1
	 IRRELEVANT 1
fayettvill
	 global 1
	 IRRELEVANT 1
vic
	 global 1
	 IRRELEVANT 1
schmaedick
	 global 2
	 IRRELEVANT 2
lespinass
	 glob

	 global 2
	 SPORTS 2
osea
	 global 2
	 SPORTS 2
sather
	 global 1
	 SPORTS 1
tatton
	 global 1
	 IRRELEVANT 1
fock
	 global 2
	 IRRELEVANT 2
roc
	 global 1
	 IRRELEVANT 1
kcb
	 global 1
	 ARTS CULTURE ENTERTAINMENT 1
gestamtmetal
	 global 1
	 IRRELEVANT 1
governmt
	 global 1
	 IRRELEVANT 1
ibngr
	 global 1
	 IRRELEVANT 1
consump
	 global 1
	 IRRELEVANT 1
lifea
	 global 1
	 IRRELEVANT 1
uness
	 global 1
	 IRRELEVANT 1
mckechnie
	 global 2
	 DOMESTIC MARKETS 2
heckuv
	 global 1
	 SPORTS 1
steppingston
	 global 1
	 SPORTS 1
shotmak
	 global 1
	 SPORTS 1
kipush
	 global 1
	 IRRELEVANT 1
bakwang
	 global 1
	 IRRELEVANT 1
sibek
	 global 2
	 IRRELEVANT 2
miba
	 global 1
	 IRRELEVANT 1
mcstay
	 global 1
	 SPORTS 1
levein
	 global 1
	 SPORTS 1
hendr
	 global 2
	 SPORTS 2
dockwork
	 global 1
	 IRRELEVANT 1
coomb
	 global 2
	 IRRELEVANT 2
mua
	 global 3
	 IRRELEVANT 3
senegal
	 global 1
	 HEALTH 1
kaolack
	 global 1
	 HEALTH 1
masserign
	 global 1
	 HEALTH 1
ndiay
	 global 1
	 HEALTH 1
stasc
	 g

	 SHARE LISTINGS 2
mainboard
	 global 1
	 SHARE LISTINGS 1
valeo
	 global 1
	 IRRELEVANT 1
denant
	 global 1
	 IRRELEVANT 1
opos
	 global 1
	 IRRELEVANT 1
kinz
	 global 3
	 IRRELEVANT 3
stoj
	 global 1
	 IRRELEVANT 1
ien
	 global 3
	 IRRELEVANT 3
daylong
	 global 1
	 SPORTS 1
jellyf
	 global 1
	 SPORTS 1
malecon
	 global 1
	 SPORTS 1
pignatiel
	 global 1
	 SPORTS 1
hallucin
	 global 1
	 SPORTS 1
nyad
	 global 1
	 SPORTS 1
poenisch
	 global 1
	 SPORTS 1
maroney
	 global 12
	 SPORTS 12
ruhrg
	 global 1
	 IRRELEVANT 1
scedul
	 global 1
	 IRRELEVANT 1
therof
	 global 1
	 IRRELEVANT 1
whitehill
	 global 1
	 SPORTS 1
hawick
	 global 1
	 SPORTS 1
chattahooch
	 global 1
	 IRRELEVANT 1
oversea
	 global 3
	 IRRELEVANT 3
baishun
	 global 1
	 IRRELEVANT 1
sos
	 global 2
	 IRRELEVANT 2
goodbod
	 global 1
	 IRRELEVANT 1
expan
	 global 1
	 IRRELEVANT 1
althlet
	 global 1
	 SPORTS 1
x100m
	 global 1
	 SPORTS 1
baily
	 global 1
	 SPORTS 1
traips
	 global 1
	 FOREX MARKETS 1
aroun
	 global 1
	 FOREX MAR

In [18]:
##look into entropy and all that jazz and do some statistics on the corpus

In [19]:
class_corpus = {}
for row in training_df.itertuples():
    words = row.article_words.split(',')
    topic = row.topic
    if topic not in class_corpus:
        class_corpus[topic] = {}
    for word in words:
        if word not in class_corpus[topic]:
            class_corpus[topic][word] = 0
        class_corpus[topic][word] += 1

geom_class_corpus = {}
for topic, dist in class_corpus.items():
    geom_class_corpus[topic] = sorted(list(dist.items()), key=lambda x:-x[1])
    #geom_class_corpus[topic] = sorted(dist.items(), key=lambda x:x[1])

#print(list(geom_class_corpus['IRRELEVANT'].items()))

In [24]:
series_list = [pd.Series(geom_class_corpus[topic], name=topic) for topic in geom_class_corpus.keys()]
print('Conditional Distributions for each Topic')
report_df = pd.concat(series_list, axis=1)
print(report_df)
k = 50
for topic, dist in geom_class_corpus.items():
    #print(topic, dist[:k])
    print(topic+":")
    size = len(dist)
    for i in range(k):
        print('\t', dist[size-i-1])

Conditional Distributions for each Topic
          FOREX MARKETS    MONEY MARKETS           SPORTS        IRRELEVANT  \
0        (dollar, 3358)     (bank, 4447)     (play, 1491)   (percent, 5945)   
1          (bank, 2178)     (rate, 3772)      (win, 1054)      (year, 5237)   
2          (rate, 1755)  (percent, 3515)      (game, 947)   (million, 4740)   
3          (trad, 1678)   (dollar, 3387)      (year, 938)      (trad, 3678)   
4        (market, 1668)   (market, 2901)     (world, 928)    (market, 3556)   
5       (percent, 1336)     (trad, 2490)     (match, 925)      (shar, 3294)   
6       (currenc, 1292)      (day, 1775)     (champ, 880)    (compan, 3190)   
7          (deal, 1208)     (deal, 1707)       (cup, 786)      (stat, 3160)   
8          (mark, 1191)     (cent, 1533)      (team, 734)      (bank, 3095)   
9            (yen, 962)  (currenc, 1472)      (scor, 707)      (pric, 2611)   
10          (cent, 953)     (mark, 1400)     (final, 703)    (govern, 2328)   
11         